In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftConfig, PeftModel

device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
user_token = os.getenv("HF_API_TOKEN")
model_name = "Qwen/Qwen2.5-7B-Instruct"
adapter_name = "JesseLiu/qwen25-7b-pagerank-partial"

peft_cfg = PeftConfig.from_pretrained(adapter_name, use_auth_token=user_token)
base_name = model_name

tokenizer = AutoTokenizer.from_pretrained(base_name, use_auth_token=user_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

new_special_tokens = [
    '<degd>', '<ddd>', '<decgd>',
    '<demgd>', '<debgd>', '<dppd>', '<dpd>'
]
tokenizer.add_special_tokens({'additional_special_tokens': new_special_tokens})

base_model = AutoModelForCausalLM.from_pretrained(
    base_name,
    torch_dtype="auto",
    use_auth_token=user_token
).to(device)

base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(
    base_model,
    adapter_name,
    torch_dtype="auto"
).to(device)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id if tokenizer.bos_token else None

model.eval()

prompt = "Please output a <degd> token now:"
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

generated_ids = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.8
)

output = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

print(output)


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 76.62it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 4 has a total capacity of 47.40 GiB of which 1.06 MiB is free. Process 3551285 has 16.86 GiB memory in use. Including non-PyTorch memory, this process has 13.65 GiB memory in use. Process 3565959 has 16.86 GiB memory in use. Of the allocated memory 13.08 GiB is allocated by PyTorch, and 92.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)